In [ ]:
#pip install pandas

In [1]:
# !pip install sentence-transformers

  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
  Using cached torch-2.0.1-cp39-cp39-win_amd64.whl (172.4 MB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached torchvision-0.15.2-cp39-cp39-win_amd64.whl (1.2 MB)


In [8]:
# pip install --user --upgrade --force-reinstall scipy

  Using cached scipy-1.11.1-cp39-cp39-win_amd64.whl (44.1 MB)
  Using cached numpy-1.25.2-cp39-cp39-win_amd64.whl (15.6 MB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
# pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the cleaned data from CSV into a pandas DataFrame
cleaned_data = pd.read_csv("C:\\Users\\Dell\\OneDrive - Northeastern University\\courses\\big data and intl analytics\\DAMG7245-Summer2023\\final project\\dataset_converted\\3d_printing\\posts_cleaned.csv")
comments_df = pd.read_csv("C:\\Users\\Dell\\OneDrive - Northeastern University\\courses\\big data and intl analytics\\DAMG7245-Summer2023\\final project\\dataset_converted\\3d_printing\\comments_cleaned.csv")
# Load the pre-trained Sentence Transformers model
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Preprocessing function to convert tags from string to list
def process_tags(Tags):
    if pd.isna(Tags):  # Handle NaN values
        return []      # Return an empty list for NaN values
    return Tags.split(',')

# Apply the preprocessing to the 'Tags' column (uppercase "T")
cleaned_data['post_tags'] = cleaned_data['post_tags'].apply(process_tags)

def generate_embeddings(text):
    embeddings = model.encode(text)
    return embeddings

def filter_data(user_input_tag):
    # Keep rows with "posttypeid" as 1 or 2
    filtered_data = cleaned_data

    # Keep only data for the user input tag
    filtered_data = filtered_data[filtered_data['post_tags'].apply(lambda tags: user_input_tag in tags)]

    return filtered_data

def topic_relevance_search(user_input, user_input_tag):
    # Filter data based on user input tag and "PostTypeId"
    filtered_data = filter_data(user_input_tag)

    # Check if there is relevant data for the user input tag
    if filtered_data.empty:
        print(f"No relevant data found for the tag '{user_input_tag}'.")
        return []

    # Concatenate relevant columns for embedding, handling NaN values
    relevant_text = filtered_data.apply(lambda row: ' '.join(filter(lambda x: pd.notna(x), [row['post_title'], row['post_body']])), axis=1)

    # Convert relevant_text to a list
    relevant_text_list = relevant_text.tolist()

    # Generate embeddings for user input and relevant data
    user_embedding = generate_embeddings([user_input])
    data_embeddings = generate_embeddings(relevant_text_list)

    # Calculate cosine similarity between user input and data embeddings
    similarity_scores = cosine_similarity(user_embedding, data_embeddings)[0]

    # Sort the data by similarity scores in descending order and select top 5 rows
    top_indices = np.argsort(similarity_scores)[::-1][:5]
    
    top_similar_topics = filtered_data.iloc[top_indices][['post_id', 'post_title', 'posttypeid', 'post_body', 'post_body_accepted']].to_dict('records')
    
    top_similarity_scores = similarity_scores[top_indices]

    return top_similar_topics, top_similarity_scores

if __name__ == "__main__":
    # Example usage:
    user_question = "what is stl file?"
    user_tag = "<3d-design>"

    similar_topics, similarity_scores = topic_relevance_search(user_question, user_tag)
    
    for i in range(len(similar_topics)):
        post_id, post_title, posttypeid, post_body, post_body_accepted = similar_topics[i]['post_id'], similar_topics[i]['post_title'], similar_topics[i]['posttypeid'], similar_topics[i]['post_body'], similar_topics[i]['post_body_accepted']
        print(f"{i+1}. {post_title} (similarity score: {similarity_scores[i]:.2f})")
        
        if pd.isna(post_body_accepted):
            print("No accepted answer")
        else:
            print(f"Accepted answer: {post_body_accepted}")
        
        if posttypeid == 1:
            print(f"Description: {post_body}")
        elif posttypeid == 2:
            print(f"Answer: {post_body}")
        
        post_comments = comments_df[comments_df['post_id'] == post_id]
        
        if not post_comments.empty:
            print("Comments:")
            for comment in post_comments['comments_text']:
                print(f"\t- {comment}")

1. Generating mold from stl file of the 3D drawing of the object (similarity score: 0.26)
No accepted answer
Description: <p>I want to know how to make a mold of a 3D design in .stl format.</p>

<p>Suppose I have a 3D partin .stl format (for e.g. a cylinder) and I want to make/design a mold for this object (i.e. the structure through which I could make the cylinder). Is there any way to do so? Are there any tools to do so?</p>

<p>My requirement is as follows: I have an .stl file of a design and need to develop the CAD files for its mold. I will then need to 3D print these molds. I would require to add a hole to pour in liquid (resin based) raw material which hardens with time. </p>

Comments:
	- What kind of mold you are looking for? for plastic extrusion or resine molding?
	- Adding a hole is trivial assuming you own a drill :-)
	- An off-the wall thought: if it were possible to modify a printer to use wax or other low-temperature melting point material,  then print the item, pack in